# Свойства многофазного потока по uniflocvba и uniflocpy

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT_fluids

In [ ]:
def create_result_df_from_vba_output(vba_result, p_bar, t_c):
    result_dict = {}
    for key, value in zip(vba_result[1],vba_result[0]):
        result_dict[key] = value
    result_dict['p_bar'] = p_bar
    result_dict['t_c'] = t_c
    this_df = pd.DataFrame(result_dict, index = [0])
    return this_df

Загрузка надстройки

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

Задание параметров

In [ ]:
gamma_oil =0.86
gamma_water = 1
gamma_gas =  1.45/1.293
rsb_m3m3 = 56
t_res_c = 40
t_c = 40
p_bar = 90
q_liq_sm3day = 40
watercut_perc = 0


keywords_python = {"gamma_oil": gamma_oil, "gamma_gas": gamma_gas, "gamma_wat":gamma_water,
                                    "rsb_m3m3": rsb_m3m3, "t_res_c": t_res_c, 'pb_bar':90}

keywords_vba = {"qliq_sm3day":q_liq_sm3day, "fw_perc":watercut_perc,
                "t_C": t_c, "gamma_gas": gamma_gas,
                "gamma_oil": gamma_oil, "gamma_wat": gamma_water, "rsb_m3m3": rsb_m3m3, "tres_C": t_res_c}



In [ ]:
gamma_gas


Настройка модели флюида

In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()
blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1
blackoil_option.z_cor_number = 1
blackoil_option.pseudocritical_temperature_cor_number = 1
blackoil_option.pseudocritical_pressure_cor_number = 1
blackoil_option.rho_gas_cor_number = 1
blackoil_option.b_gas_cor_number = 1 - 1
blackoil_option.mu_dead_oil_cor_number = 2
blackoil_option.sigma_oil_gas_cor_number = 2
blackoil_option.sigma_wat_gas_cor_number = 1

#blackoil_option.rs_cor_number =2
#blackoil_option.pb_cor_number = 3

blackoil_option.set_vba_preset()


python_fluid = BlackOil_model.Fluid(**keywords_python, option = blackoil_option)

Инициализация многофазного потока

In [ ]:
multiphase_flow = PVT_fluids.FluidFlow(python_fluid)
multiphase_flow.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow.fw_on_surface_perc  =watercut_perc

In [ ]:
import scipy.optimize as sp  # модуль для решения уравения
import numpy as np
def coef_mt(t_k, rho_deadoil_kgm3, gamma_gas):
    return 1 + 0.029 * (t_k - 293) * (rho_deadoil_kgm3 * gamma_gas * 10 ** (-3) - 0.7966)
def coef_dt(t_k, rho_deadoil_kgm3, gamma_gas):
    return 10 ** (-3) * rho_deadoil_kgm3 * gamma_gas * (4.5 - 0.00305 * (t_k - 293)) - 4.785
def coef_rp(p_mpa, pb_mpa):
    return (1 + np.log10(p_mpa)) / (1 + np.log10(pb_mpa)) - 1
def calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):

    #rsb_m3m3 = 50
    rp = coef_rp(p_mpa, pb_mpa)
    #print(f"rp={rp}")
    dt = coef_dt(t_k, rho_deadoil_kgm3, gamma_gas)
    #print(f"dt={dt}")
    mt = coef_mt(t_k, rho_deadoil_kgm3, gamma_gas)
    #print(f"mt={mt}")
    gas_liberated_m3t = rsb_m3m3 * rp * mt * (dt*(1+rp)-1)
    gas_dissolved_m3t = rsb_m3m3 * mt - gas_liberated_m3t
    
    if p_mpa >= pb_mpa:
        if return_m3m3:
            return 0, rsb_m3m3 * mt * rho_deadoil_kgm3 / 1000
    if return_m3m3:
        return gas_liberated_m3t * rho_deadoil_kgm3 / 1000, gas_dissolved_m3t * rho_deadoil_kgm3 / 1000
        #return gas_liberated_m3t, gas_dissolved_m3t

    return gas_liberated_m3t, gas_dissolved_m3t

def calc_gas_for_fsolve(rsb_m3m3_real, t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):
    gas_dissolved_m3m3 =calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, pb_mpa, pb_mpa, rsb_m3m3_real, return_m3m3 = True)[1]
    return (gas_dissolved_m3m3 - rsb_m3m3)**2

def calc_gas_with_fsolve(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, rsb_m3m3, return_m3m3 = True):
    method = 'excitingmixing'
    answer = sp.root(calc_gas_for_fsolve, rsb_m3m3, args = (t_k, rho_deadoil_kgm3, gamma_gas,
                                                            pb_mpa, pb_mpa, rsb_m3m3, return_m3m3), tol = 0.01)
    answer = answer.x[0]
    return calc_gas(t_k, rho_deadoil_kgm3, gamma_gas, p_mpa, pb_mpa, answer, return_m3m3)
    

In [ ]:
def calc_pb(pb_mpa, rho_deadoil_kgm3, rsb_m3m3, tres_k, t_k):
    gi = rsb_m3m3 * 10**3 * 273 / 293 / rho_deadoil_kgm3
    pbt_mpa = pb_mpa - (tres_k - t_k) / (9.157 + 701.8 / (gi * (0.4 - 0.8* 0.08)))
    return pbt_mpa
    

In [ ]:
calc_pb(9, 860, 56, 313, 312.8)

In [ ]:
calc_gas(300.5, 868, 1.119, 5.5, 8.95, 55.6)

In [ ]:
calc_gas_with_fsolve(300.5, 868, 1.119, 5.5, 8.95, 55.6)

Расчет

In [ ]:
python_flow_model_db = data_workflow.Data()
python_flow_model_db.clear_data()
vba_result_df = None
gas_liberated_m3m3, gas_dissolved_m3m3 = [], []
for p_bar in range(1, 120, 4):
    p_bar = p_bar + 0.1
    multiphase_flow.calc(p_bar, t_c)
    p_atm = uc.bar2atm(p_bar)
    python_flow_model_db.get_data(multiphase_flow, object_name='python_flow_model')
    vba_result = uniflocvba.MF_all_mf(p_atma = p_atm, **keywords_vba)
    
    this_gas_liberated_m3m3, this_gas_dissolved_m3m3 = calc_gas(multiphase_flow.fl.t_k,
                                                               multiphase_flow.fl.rho_oil_stkgm3,
                                                               multiphase_flow.fl.gamma_gas,
                                                               multiphase_flow.fl.p_mpa, 
                                                               multiphase_flow.fl.pb_bar / 10, 
                                                               multiphase_flow.fl.rsb_m3m3,
                                                               True)
    gas_liberated_m3m3.append(this_gas_liberated_m3m3)
    gas_dissolved_m3m3.append(this_gas_dissolved_m3m3)
    this_vba_result_df = create_result_df_from_vba_output(vba_result, p_bar, t_c)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

Объединение результатов

In [ ]:
result_unifloc_python = python_flow_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_flow_model.p_bar']
result_unifloc_python.index.name = 'p_bar'

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df['mn_kgsec'] = vba_result_df['mw_kgsec'] + vba_result_df['mg_kgsec'] + vba_result_df['mo_kgsec']
vba_result_df['liquid_content_d'] = 1 - vba_result_df['gas_fraction_d']
vba_result_df = vba_result_df.add_prefix('vba_flow_model.')
all_result = result_unifloc_python.join(vba_result_df)
df = pd.DataFrame({'gas_liberated_m3m3m':gas_liberated_m3m3,
                   'gas_dissolved_m3m3m':gas_dissolved_m3m3},
                   all_result.index)
all_result = all_result.join(df)

Построение графиков

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['dissolved_gas_sc_m3m3'],
                                              ['liberated_gas_sc_m3m3'],
                                              ['m3m3m'],], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Объем растворенного и выделившегося газа', 
                                x_name = 'Давление, бар', 
                                y_name = 'Объем газа, м3/м3',
                                 traces_names = ['Объем растворенного газа по Стендингу, м3/м3', 
                                                'Объем выделившегося газа по Стендингу, м3/м3', 
                                                'Объем выделившегося газа по Мищенко, м3/м3',
                                                'Объем растворенного газа по Мищенко, м3/м3']
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['gas_fraction'], ['liquid_content']], ['vba','fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  
                                plot_name='Объемная доля жидкости и газа', 
                                x_name = 'Давление, бар', 
                                y_name = 'д.ед.',
                                 traces_names = ['Объемная доля газа в потоке ГЖС, д.ед.', 
                                                'Объемная доля жидкости в потоке ГЖС, д.ед.' 
                                                ])

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['pb_bar'], ['cor_number'], swap_xy=False, reversed_y=False,  
                                 plot_name='Давление насыщения', 
                                x_name = 'Давление, бар',
                                y_name = 'Давление насыщения, бар',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Давление насыщения по исходным данным, бар'])

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_gas_cp']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Вязкость газа', 
                                 x_name = 'Давление, бар',
                                y_name = 'Вязкость газа, сПуаз',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['z']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,
                                 plot_name='Коэффициент сверхсжимаемости', 
                                 x_name = 'Давление, бар',
                                y_name = 'Коэффициент сверхсжимаемости',
                                traces_names = ['Давление насыщения по Стендингу, бар', 
                                                'Вязкость нефти, сПуаз']
                        )

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil']], ['cor_number', 'cal', 'stkgm', 'b_oil_b'], swap_xy=False, reversed_y=False,
                                 plot_name='Объемный коэффициент нефти', 
                                 x_name = 'Давление, бар',
                                y_name = 'Объемный коэффициент нефти, м3/м3',
                                #traces_names = ['Давление насыщения по Стендингу, бар', 
                                #                'Вязкость нефти, сПуаз']
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.mu'],['fl.mu']], ['dead','vba','oil_b','cor_number'], swap_xy=False, reversed_y=False, 
                                plot_name='Вязкость ГЖС и составляющих', 
                                x_name = 'Давление, бар', 
                                y_name = 'Вязкость, сПуаз',
                                 traces_names = ['Вязкость жидкости, сПуаз', 
                                                'Объемная вязкость ГЖС, сПуаз', 
                                                'Вязкость нефти, сПуаз',
                                                'Вязкость газа, сПуаз',
                                                'Вязкость воды, сПуаз']
                                 )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['flow_model.rho'],['fl.rho']], ['dead','vba','oil_b','cor_number', 'stkgm'], swap_xy=False, reversed_y=False, 
                                plot_name='Плотность ГЖС и составляющих', 
                                x_name = 'Давление, бар', 
                                y_name = 'Плотность, кг/м3',
                                traces_names = ['Плотность жидкости, кг/м3', 
                                               'Объемная плотность ГЖС, кг/м3', 
                                               'Плотность нефти, кг/м3',
                                               'Плотность газа, кг/м3',
                                               'Плотность воды, кг/м3']
                                
                                )

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["heatcap"], ["cliq_JkgC"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["sigma"]], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["kgsec"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

# Суммарный массовый расход смеси должен быть постоянным

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'oil']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'wat']], ['fl.','cor_number', 'mass_fraction', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'gas']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs'], ['liberated_gas'], ['dissolved_gas_sc_m3m3']], ['cor_number', 'cal', 'rsw'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liq', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mix', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  ['fw'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liquid_content']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rs']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['rho_oil']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['b_oil']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mu_oil_cp']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')